In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
import numpy as np
pd.set_option('display.max_columns',120)
def delUnname0(df):
    df = df.drop('Unnamed: 0',axis=1)
    return df
def xgb_eval_custom_r(y_pred, dtrain):
    y_true = dtrain.get_label()
    up = np.sum(y_pred - y_true)
    down = np.sum(y_pred + y_true)
    whole = np.abs(up/down)
    Loss = whole/len(y_true)
    #here only predict one shop 
    return 'Loss', Loss
#preprocessing normailze?No.Do a one hot encoding~~Because the data is so small~~
def TimeFeat(df):
    columns = df.columns
#     df['month'] = pd.Index(df[columns[1]]).month
#     df['day'] = pd.Index(df[columns[1]]).day
    df['dayofweek'] = pd.Index(df[columns[1]]).dayofweek
#     dummies_month = pd.get_dummies(df['month'],prefix='month')
#     dummies_day = pd.get_dummies(df['day'],prefix='day')
    dummies_dayofweek = pd.get_dummies(df['dayofweek'],prefix='dayofweek')  
#     df_dummies = df.join([dummies_month,dummies_day,dummies_dayofweek],how='outer')
#     df_dummies = df_dummies.drop(['month','day','dayofweek'],axis=1)

    df_dummies = df.join(dummies_dayofweek,how='outer')
    df_dummies = df_dummies.drop(['dayofweek'],axis=1)
    return df_dummies  

In [3]:
Customer_Flow = pd.read_csv("Customer_Flow.csv")
Customer_Flow = delUnname0(Customer_Flow)
Customer_Flow['data'] = pd.to_datetime(Customer_Flow['data'])

In [4]:
# 这里有个问题如果拿10-18~10-31的数据作为测试时刚好商店1824是没有数据的
startt = pd.to_datetime("2016-10-18")
endt = pd.to_datetime("2016-10-31")
p_startt = pd.to_datetime("2016-11-01")
p_endt = pd.to_datetime("2016-11-14")
p_time =pd.date_range(start=p_startt,end=p_endt,freq='D')
p_shop_id = range(1,2001,1)
shopid = []
p_t = []
for shop_id in p_shop_id:
    for t in p_time:
        shopid.append(shop_id)
        p_t.append(t)
predict ={'data':p_t,'shop_id':shopid}
predict = pd.DataFrame(predict)

copy_Customer_Flow = Customer_Flow
all_data  = copy_Customer_Flow.merge(predict,how='outer')
all_data = TimeFeat(all_data)

In [5]:
# this parameter is basic on shop 1780 choosed ramdomly
#best parameter max_depth=6,gama=0.07,min_child_weight=12,'subsample': 0.4, 'colsample_bytree': 0.9,'reg_alpha': 0.06
p_result = []
empty_test_id = []
for shop_id,eachShop in all_data.groupby(['shop_id']):
    test_df =eachShop[((eachShop['data'] >= startt) & (eachShop['data'] <= endt))]
#     if test data is null,i will use some data to fill with actually,only 1824 is empty
    if test_df.empty == True:   
        print 'empty id:',shop_id
        empty_test_id = empty_test_id.append(shop_id)
        cols = []
        for col in test_df.columns:
            cols.append(col)
        print cols
        value = [[0,0,shop_id,0,0,0,0,0,0,0]]
        print np.shape(value)
        value = np.array(value)
        defaultLine = pd.DataFrame(value,columns = cols)
        test_df = test_df.merge(defaultLine,how='outer')
    train_df =eachShop[(eachShop['data'] < startt)]
    predict_df =eachShop[((eachShop['data'] >= p_startt) & (eachShop['data'] <= p_endt))]
    ToDrop = ['Num','data','shop_id']
    #     Turn data to xgboost format
    train_xgb = xgb.DMatrix(data=train_df.drop(ToDrop,axis=1)
                            ,label=train_df['Num'])
    valid_xgb = xgb.DMatrix(data=test_df.drop(ToDrop,axis=1)
                            ,label=test_df['Num'])
    predict_xgb = xgb.DMatrix(data = predict_df.drop(ToDrop,axis = 1))
    params = {
      'objective': 'reg:linear'
      ,'eta': 0.1
      ,'max_depth': 6
      , 'subsample': 0.4
      , 'colsample_bytree': 0.9
      ,'min_child_weight': 12
      ,'gamma': 0.07
      , 'seed': 10
    ,'reg_alpha': 0.06
    }       
    evallist = [(train_xgb, 'train'), (valid_xgb, 'valid')]
    model = xgb.train(params.items()
                      , dtrain=train_xgb
                      , num_boost_round=10000
                      , evals=evallist
                      , early_stopping_rounds=20
                      , maximize=False
                      , verbose_eval=10
                      , feval=xgb_eval_custom_r
                  )
    print "now it is:",shop_id
    print ('get info from model')                  
    print (model.best_score, model.best_iteration,model.best_ntree_limit)

    result = pd.DataFrame(model.predict(predict_xgb,ntree_limit=model.best_ntree_limit))  
    result = result.rename(columns={0:'Num'})
    each_line = []
    each_line.append(shop_id)
    for Num in result.Num:
        each_line.append(Num) 
    p_result.append(each_line)  

[0]	train-rmse:225.93	valid-rmse:202.726	train-Loss:0.00219	valid-Loss:0.057517
Multiple eval metrics have been passed: 'valid-Loss' will be used for early stopping.

Will train until valid-Loss hasn't improved in 20 rounds.
[10]	train-rmse:100.772	valid-rmse:67.941	train-Loss:0.000504	valid-Loss:0.011089
[20]	train-rmse:70.3316	valid-rmse:34.4472	train-Loss:0.00015	valid-Loss:0.001576
[30]	train-rmse:65.9565	valid-rmse:34.4074	train-Loss:4.6e-05	valid-Loss:0.001194
[40]	train-rmse:65.5907	valid-rmse:34.2614	train-Loss:2.5e-05	valid-Loss:0.001751
Stopping. Best iteration:
[25]	train-rmse:67.4502	valid-rmse:33.8565	train-Loss:9.4e-05	valid-Loss:6.5e-05

now it is: 1.0
get info from model
(6.5e-05, 25, 26)
[0]	train-rmse:208.209	valid-rmse:86.59	train-Loss:0.002481	valid-Loss:0.066735
Multiple eval metrics have been passed: 'valid-Loss' will be used for early stopping.

Will train until valid-Loss hasn't improved in 20 rounds.
[10]	train-rmse:180.409	valid-rmse:21.3446	train-Loss:0.00052

In [6]:
# here ,i occur some problem that some predict value is negative (around 4 values),and i comple it to be non_negative with abs function
p_result = abs(pd.DataFrame(p_result).astype(int)) 
p_result.to_csv('prediction2.csv',header=False,index=False,encoding='utf-8')

In [8]:
p_result

0    1    2    3    4    5    6    7    8    9    10   11   12   13  \
0        1  214  225  221  247  233  214  211  214  225  221  247  233  214   
1        2   70   77   73   84  141  142   73   70   77   73   84  141  142   
2        3   75   75   75   75   78   76   74   75   75   75   75   78   76   
3        4  123  123  123  123  123  123  123  123  123  123  123  123  123   
4        5  167  170  159  134  155  135  166  167  170  159  134  155  135   
5        6   61   60   57   64   62   59   58   61   60   57   64   62   59   
6        7  120  121  117  137  140  151  114  120  121  117  137  140  151   
7        8   63   63   63   63   63   63   63   63   63   63   63   63   63   
8        9  172  171  171  179  198  204  177  172  171  171  179  198  204   
9       10   40   40   39   40   40   40   40   40   40   39   40   40   40   
10      11   57   60   63   67   44   39   57   57   60   63   67   44   39   
11      12   46   53   66   59   73   69   53   46   53   66   59   73   69   
12      13  162  163  157  167  155  146  159  162  163  157  167  155  146   
13      14   55   55   52   56   54   49   55   55   55   52   56   54   49   
14      15  109  109  109  109  109  109  109  109  109  109  109  109  109   
15      16   37   41   41   54   48   51   40   37   41   41   54   48   51   
16      17   74   74   74   82   86   85   64   74   74   74   82   86   85   
17      18   62   62   62   62   62   62   62   62   62   62   62   62   62   
18      19  136  139  138  146  177  192  139  136  139  138  146  177  192   
19      20   96   98  108  100   80   76  110   96   98  108  100   80   76   
20      21   69   69   74   89  107  111   69   69   69   74   89  107  111   
21      22   69   71   68   83   77   73   77   69   71   68   83   77   73   
22      23   37   37   37   37   16   16   37   37   37   37   37   16   16   
23      24   55   67   63   74  100   98   54   55   67   63   74  100   98   
24      25   94   92   92  104  108   99   85   94   92   92  104  108   99   
25      26   48   47   52   47   82   82   48   48   47   52   47   82   82   
26      27   63   63   63   64   63   60   65   63   63   63   64   63   60   
27      28   54   56   55   62   68   70   48   54   56   55   62   68   70   
28      29   59   62   57   80  105   92   59   59   62   57   80  105   92   
29      30   54   49   42   52   51   38   51   54   49   42   52   51   38   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
1970  1971  144  141  130  155   99   94  144  144  141  130  155   99   94   
1971  1972   59   60   67   79  112  104   56   59   60   67   79  112  104   
1972  1973   43   44   44   43   31   32   46   43   44   44   43   31   32   
1973  1974  109  101   99  112   67   66   96  109  101   99  112   67   66   
1974  1975  121  124  132  171  184  184  123  121  124  132  171  184  184   
1975  1976   98   98   86   99  102   99   97   98   98   86   99  102   99   
1976  1977  100   99   97   99   98   92  104  100   99   97   99   98   92   
1977  1978   96   97   97  106  100   99   90   96   97   97  106  100   99   
1978  1979   65   65   65   64   66   65   64   65   65   65   64   66   65   
1979  1980   48   48   47   50   60   56   46   48   48   47   50   60   56   
1980  1981   53   50   44   42   39   40   49   53   50   44   42   39   40   
1981  1982   64   64   64   65   64   57   64   64   64   64   65   64   57   
1982  1983   55   55   55   55   55   55   55   55   55   55   55   55   55   
1983  1984   58   58   58   58   58   58   58   58   58   58   58   58   58   
1984  1985  258  299  322  420  325  300  264  258  299  322  420  325  300   
1985  1986   80   83   84   90   94   92   77   80   83   84   90   94   92   
1986  1987   39   41   41   51   67   68   40   39   41   41   51   67   68   
1987  1988  159  163  169  173  226  233  168  159  163  169  173  226  233   
1988  1989   45   54   54   78   82   86   54   45   54   